File handling and PReprocessing

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,BatchNormalization,Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import img_to_array,load_img
import cv2
import os

In [145]:
path1=r"c:\users\kumar\Documents\deep\Normalvideos"
path2=r"c:\users\kumar\documents\deep\shoplifting"
path3=r"c:\users\kumar\documents\deep\stealing"

In [147]:
file1=os.listdir(path1)
file2=os.listdir(path2)
file3=os.listdir(path3)

In [213]:
x_data=[]
y_data=[]
def fun(path,file,y_label):
    for i in range(0,1000):
        img=cv2.imread(path+"\\"+file[i])
        img=cv2.resize(img,(224,224))
        #gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        array=img_to_array(img)
        x_data.append(array)
        y_data.append(y_label)

In [215]:
fun(path1,file1,[1,0,0])
fun(path2,file2,[0,1,0])
fun(path3,file3,[0,0,1])

In [217]:
import numpy as np
x=np.array(x_data)
y=np.array(y_data)
#np.save("x3.npy",x)
#np.save("y3.npy",y)

In [116]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,BatchNormalization,Dropout,Flatten
from keras.optimizers import SGD,Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.regularizers import l2

In [118]:
x=np.load("x3.npy")
y=np.load("y3.npy")

In [219]:
np.shape(x)

(15000, 224, 224, 1)

In [221]:
x_transform=x.astype("float32")/255

In [223]:
import random
paired = list(zip(x_transform, y))
random.shuffle(paired)
x_train, y_train = zip(*paired)
x_train = np.array(x_train)
y_train = np.array(y_train)

In [226]:
from sklearn.model_selection import train_test_split

In [228]:
from sklearn.model_selection import train_test_split

# Assuming x_data and y_data are your full dataset
x_train, x_temp, y_train, y_temp = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
"""
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())"""
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(96, kernel_regularizer=l2(0.01),activation='relu'))
model.add(Dense(3, activation='softmax'))
opt=SGD(learning_rate=0.001,momentum=0.9)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

#TRaining

In [238]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from keras.regularizers import l2
from keras.optimizers import SGD,Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler

def scheduler(epoch, lr):
    if epoch > 2:
        return lr * 0.01 
    return lr

model = Sequential()
model.add(Input(shape=(224, 224, 1)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(96, kernel_regularizer=l2(0.01), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Optimizer
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
lr_schedule = LearningRateScheduler(scheduler)

In [239]:

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

x_train shape: (12000, 224, 224, 1)
y_train shape: (12000, 3)


In [242]:
print(y_train[:5])  # Check the first few labels
print(y_train.dtype)  # Ensure it's float32
y_train=y_train.astype("float32")

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
float32


In [244]:
model.fit(
    x_train,
    y_train,
    epochs=10,  # Increased epochs for better learning, paired with early stopping
    batch_size=32,  # Kept the batch size as it is
    validation_data=(x_val, y_val),  # Adding validation data to monitor overfitting
    callbacks=[early_stop, lr_schedule],  # Added callbacks for early stopping and learning rate scheduling
    verbose=1  # To display the training process
)

Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 294s 500ms/step - accuracy: 0.9521 - loss: 8.5264 - val_accuracy: 0.6313 - val_loss: 29.8700 - learning_rate: 0.0010
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 173s 460ms/step - accuracy: 0.9915 - loss: 7.0452 - val_accuracy: 1.0000 - val_loss: 8.3619 - learning_rate: 0.0010
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 173s 459ms/step - accuracy: 0.9941 - loss: 6.3275 - val_accuracy: 0.9980 - val_loss: 5.1839 - learning_rate: 0.0010
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 168s 449ms/step - accuracy: 0.9952 - loss: 5.1733 - val_accuracy: 0.9987 - val_loss: 4.8750 - learning_rate: 1.0000e-05


model.summary()

In [60]:
y_pred=model.predict(x_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step


In [41]:
model.evaluate(x_test,y_test)

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 175ms/step - accuracy: 0.2824 - loss: 0.8734


[0.8657683730125427, 0.2916666567325592]

In [29]:
#model.save("Deep_learning_3cat_new.h5")

In [254]:
model.save("Deep_learning_3cat_adam_gray.h5")

In [ ]:
model

In [250]:
from tensorflow.keras.models import load_model as lm

In [252]:
model=lm("Deep_learning_3cat_adam_gray.h5")

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'Deep_learning_3cat_adam_gray.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [57]:
model1.evaluate(x_test,y_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.9092 - loss: 1.6271


[1.5015525817871094, 0.9133333563804626]

In [59]:
y_pred=model1.predict(x_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step


benchmark

In [133]:
def funct(l):
    y=[]
    for i in l:
        y.append(np.argmax(i))
    return y

In [135]:
import pandas as pd
data=pd.DataFrame()
data["y"]=funct(y_test)
data["y_pred"]=funct(y_pred)

In [145]:
data.to_excel("model_benchmark.xlsx")

In [25]:
from tensorflow.keras.models import load_model as lm

prediction 


In [131]:
img=cv2.imread(r"C:\Users\kumar\Downloads\test7.jpeg")# testing frame
img1=cv2.resize(img, (64,64))
gray_image = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
array_img=img_to_array(gray_image)

print(np.shape(array_img))
array_img=array_img.reshape(1,64,64,1)
array_sen_img=array_img/255.0
model.predict(array_sen_img)

(64, 64, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step


array([[9.9992979e-01, 4.3942837e-05, 2.6234708e-05]], dtype=float32)

In [246]:
import cv2## testing the video clip
from keras.preprocessing.image import img_to_array,load_img
from keras.models import load_model
import numpy as np
video_path = r"C:\Users\kumar\Documents\test.mp4"
vid = cv2.VideoCapture(video_path)
a=load_model(r"Deep_learning_3cat_adam.h5")
name=["Normal","Shoplifiting","Stealing"]

while(True):
    ret, frame = vid.read()
    frame=cv2.resize(frame,(224,224))
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    array=img_to_array(gray_image)
    
    array=array.reshape(1,224,224,1)
    xdata=np.array(array)
    xdata=xdata.astype("float32")
    xdata=xdata/255.0
    y=a.predict(xdata)

    text=name[np.argmax(y[0])]
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2
    frame = cv2.putText(frame, text, org, font,fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow('frame', frame)
    print(text)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
vid.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Normal
1/1 ━━━━━

In [43]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from keras.callbacks import EarlyStopping, LearningRateScheduler


def scheduler(epoch, lr):
    if epoch > 2:
        return lr * 0.03  # Reduce learning rate by 10% after the 2nd epoch
    return lr


# Load ResNet50 pretrained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model (initial training)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
model_resnet = Sequential([
    base_model,
    Flatten(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Assuming 3 classes
])

early_stop = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
lr_schedule = LearningRateScheduler(scheduler)
# Compile the model
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
model_resnet.fit(
    x_train,
    y_train,
    epochs=10,  # Increased epochs for better learning, paired with early stopping
    batch_size=32,  # Kept the batch size as it is
    validation_data=(x_val, y_val),  # Adding validation data to monitor overfitting
    callbacks=[early_stop, lr_schedule],  # Added callbacks for early stopping and learning rate scheduling
    verbose=1  # To display the training process
)

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 81s 954ms/step - accuracy: 0.5472 - loss: 4.6137 - val_accuracy: 0.9900 - val_loss: 0.2283 - learning_rate: 0.0010
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 68s 904ms/step - accuracy: 0.9671 - loss: 0.1778 - val_accuracy: 0.9933 - val_loss: 0.1634 - learning_rate: 0.0010
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 68s 914ms/step - accuracy: 0.9885 - loss: 0.1064 - val_accuracy: 0.9967 - val_loss: 0.0946 - learning_rate: 0.0010
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 68s 901ms/step - accuracy: 0.9993 - loss: 0.0540 - val_accuracy: 0.9967 - val_loss: 0.0889 - learning_rate: 3.0000e-05


In [47]:
model_resnet.evaluate(x_test,y_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 667ms/step - accuracy: 1.0000 - loss: 0.0564


[0.057322707027196884, 1.0]

In [55]:
model_resnet.save("model_resnet.h5")

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Flatten
from tensorflow.keras.layers import Dense, Dropout, Reshape, Bidirectional, LSTM

# Initializing the model
model = Sequential()

# Conv2D layers for spatial feature extraction
model.add(Input(shape=(224, 224, 3)))  # Input shape for RGB images (224x224x3)
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Flatten the Conv2D outputs
model.add(Flatten())

# Reshaping for sequential processing with BiLSTM
model.add(Reshape((128, -1)))  # Reshape flattened data into a sequence (128 timesteps)

# BiLSTM layers for sequential pattern recognition
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32, return_sequences=False)))
model.add(Dropout(0.5))

# Fully connected layers for classification
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))  # Sigmoid for binary classification

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Example for training
model.fit(
    x_train,
    y_train,
    epochs=10,  # Increased epochs for better learning, paired with early stopping
    batch_size=32,  # Kept the batch size as it is
    validation_data=(x_val, y_val),  # Adding validation data to monitor overfitting
    callbacks=[early_stop, lr_schedule],  # Added callbacks for early stopping and learning rate scheduling
    verbose=1  # To display the training process
)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 112, 112, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 56, 56, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_2 (Reshape)                  │ (None, 128, 784)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 128, 128)            │         434,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ (None, 64)                  │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │             771 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 587,459 (2.24 MB)

 Trainable params: 587,011 (2.24 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 68s 663ms/step - accuracy: 0.8760 - loss: 0.3264 - val_accuracy: 0.3083 - val_loss: 0.8834 - learning_rate: 0.0010
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 36s 594ms/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.3083 - val_loss: 1.0362 - learning_rate: 0.0010
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 34s 568ms/step - accuracy: 1.0000 - loss: 9.0191e-04 - val_accuracy: 0.3083 - val_loss: 1.1387 - learning_rate: 0.0010


In [21]:
model.save("deep_cnn_bilstm.h5")